In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.vision import *

In [ ]:
bs = 64

In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/sample_submission.csv")

test_img = ImageList.from_df(test_df, path='../input/test', folder='test')
train_img = (ImageList.from_df(train_df, path='../input/train', folder='train')
        .random_split_by_pct(0.2)
        .label_from_df()
        .add_test(test_img)
        .transform(get_transforms(flip_vert=True), size=128)
        .databunch(path='.', bs=64)
        .normalize(imagenet_stats)
       )

In [ ]:
train_img.show_batch(rows=3, figsize=(3,4))

In [ ]:
learn = create_cnn(train_img,models.resnet34, metrics=[error_rate, accuracy])

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit(epochs=5,lr=2e-6)

In [ ]:
preds,_ = learn.get_preds(ds_type=DatasetType.Test)
preds.shape

In [ ]:
classes = preds.argmax(1)

test_df.has_cactus = classes
test_df.head()

In [ ]:
test_df.to_csv('submission_new.csv', index=False)